In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import sys
import re
import string
import unicodedata

# Load the new dataset
file_path = 'dataset/review_bank_digital_googleplaystore_cleaned.csv'
df = pd.read_csv(file_path)



In [10]:
import nltk
import json
import unicodedata
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from textblob import TextBlob, Word
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
snowball = nltk.stem.SnowballStemmer('english')
porter_stemmer = PorterStemmer()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
import re
import pandas as pd
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk


In [5]:

# Ensure NLTK data is downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Load Javanese and Sundanese stopwords
javanese_file_path = 'stopwords/javanese_stopwords.csv'
sundanese_file_path = 'stopwords/stopwords_sunda.csv'

# Load Javanese stopwords
javanese_stopwords_df = pd.read_csv(javanese_file_path, header=None, names=['stopword'])
javanese_stopwords = javanese_stopwords_df['stopword'].tolist()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Load Sundanese stopwords
sundanese_stopwords_df = pd.read_csv(sundanese_file_path, header=None, names=['stopword'])
sundanese_stopwords = sundanese_stopwords_df['stopword'].tolist()


In [7]:

# Combine all stopwords
combined_stopwords = set(stopwords.words('indonesian') + javanese_stopwords + sundanese_stopwords)


In [8]:
combined_stopwords

{'ngagunakeun',
 'ngayakinkeun',
 'begitu',
 'mirip',
 'Pendhaftaran',
 'ngarasa',
 'tambahnya',
 'tiap',
 'patarosan',
 'hartosna',
 'secara',
 'jauh',
 'tidaklah',
 'bukankah',
 'soal',
 'hartina',
 'katanya',
 'nelepon',
 'hanyalah',
 'saméméhna',
 'berkata',
 'kecil',
 'utawa',
 'ditémbongkeun',
 'sampeyan',
 'berapa',
 'walau',
 'iraha',
 'mengibaratkan',
 'wasnya',
 'padeukeut',
 'terhadap',
 'Ing',
 'dumasarkeun kana',
 'gunakan',
 'diperlukeun',
 'misalnya',
 'berakhir',
 'wow',
 'kelihatannya',
 'seluruh',
 'semula',
 'jumlah',
 'demi',
 'saja',
 'ieu anjeunna',
 'dimulainya',
 'jelaskan',
 'mulanya',
 'anjeunna némbongkeun',
 'henteu aya manpaatna',
 'didatangkan',
 'sababaraha kali',
 'ngetik',
 'lanjutnya',
 'pihak',
 'sekali-kali',
 'tahun',
 'lumangsung',
 'nunjuk kaluar',
 'terlebih',
 'nantinya',
 'pentingnya',
 'disebutkeun',
 'dikarenakan',
 'sebutlah',
 'berkali-kali',
 'manakala',
 'diberikan',
 'tinggi',
 'mampu',
 'Nganti',
 'komprehensif',
 'meski',
 'meureun',
 

In [11]:
# Load the slang normalization lexicon
lexicon_file_path = 'kamus/nasalsabila_kamus-alay/_json_colloquial-indonesian-lexicon.txt'
with open(lexicon_file_path, 'r') as file:
    slang_dict = json.load(file)
# slang_dict
    
# Load colloquial dictionary
# with open("path_to_your_colloquial_dict/_json_colloquial-indonesian-lexicon.txt") as f:
    # data = f.read()
# Reconstruct data as a dictionary
# slang_dict = json.loads(data)



In [12]:
# Define the text preprocessing function
def text_preprocessing(text):
    
    text = text.lower() # Lowercase the text
    text = re.sub('[-+]?[0-9]+', ' ', text) # Remove numbers
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text) # Remove URLs
    text = re.sub('\S*@\S*\s?', ' ', text) # Remove email addresses
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text) # Remove punctuation and symbols
    text = re.sub(r'(\w)\1{2,}', r'\1\1', text) # Correct words with three or more repeated characters (e.g., loooove -> love)
    text = re.sub(' +', ' ', text).strip() # Remove extra whitespaces
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') # Normalize Unicode characters to ASCII
    
    # Replace slang words with their formal equivalents
    words = text.split()
    normalized_words = [slang_dict[word] if word in slang_dict else word for word in words]
    normalized_text = ' '.join(normalized_words)
    
    # Tokenize words
    word_tokens = word_tokenize(normalized_text)
    
    # Remove stopwords
    clean_words = [word for word in word_tokens if word not in combined_stopwords]
    
    # Join cleaned words back into a string
    clean_text = ' '.join(clean_words)
    
    return clean_text

<>:7: SyntaxWarning: invalid escape sequence '\S'
<>:7: SyntaxWarning: invalid escape sequence '\S'
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24440\2647155504.py:7: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('\S*@\S*\s?', ' ', text) # Remove email addresses


In [35]:
def text_preprocessingx(text):
    text = text.lower()                                   # Lowercase all sentences
    text = re.sub('[-+]?[0-9]+', ' ', text)               # Remove numbers 
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)    # Remove URLs
    text = re.sub('\S*@\S*\s?', ' ', text)                # Remove email
    text = re.sub(r'[^\w\s]', ' ', text)                  # Remove punctuation
    text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text)  # Remove symbols
    text = re.sub(r'([a-zA-Z])\1\1','\\1', text)          # Correct three or more repeated characters
    text = re.sub(' +',' ', text)                         # Remove multiple whitespaces
    text = re.sub(r'^[ ]|[ ]$','', text)                  # Remove leading and trailing spaces
    
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') # Remove non-ascii characters
    
    # Replace slang words with their formal equivalents
    words = text.split()
    normalized_words = [slang_dict[word] if word in slang_dict else word for word in words]
    normalized_text = ' '.join(normalized_words)
    
    word_tokens = word_tokenize(normalized_text) # Tokenize words
    
    # Remove stopwords
    clean_words = [word for word in word_tokens if word not in combined_stopwords]
    
    clean_text = ' '.join(clean_words)
    
    return clean_text


<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_7180\502333008.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('\S*@\S*\s?', ' ', text)                # Remove email


In [43]:

# Example usage
# sample_text = "Ini adalah contoh teks yang akan diolah dengan stopwords dari bahasa Indonesia, Jawa, dan Sunda."
# sample_text = "Aku pesen ikan bakar sama sambal terasi. Enak banget lho sambal terasi di sini ya. Ngopi neng kene saka cedhak dheweke rampung yo asyik. karep kelakon arep Bisa ngobrol santai karo kanca-kanca"
# cleaned_text = text_preprocessing(sample_text)
# print(cleaned_text)


In [13]:
sample_text = "Ini g nyangka nyebur dmn gak kebalas kejawab ngapain gue gini aja gini gitu tp blm dpt atas nama sape napa aja gak biasa klo begitu aja adalah pengen bener2 met makan ya ges contoh kereeennnn abizzzz enggak gk banget gtu teks gemess yang akan diolah dengan stopwords dari bahasa Indonesia, Jawa, dan Sunda."
cleaned_text = text_preprocessing(sample_text)
print(cleaned_text)

menyangka mencebur terbalas terjawab mengapai gue nama sape kalo pengin selamat makan ya ges contoh keren abizz teks gemas diolah stopwords bahasa indonesia jawa sunda


In [14]:
# Apply the text preprocessing function to the 'content' column
df['cleaned_content_multi'] = df['content'].apply(text_preprocessing)

# Display the first few rows of the cleaned text
df[['content', 'cleaned_content_multi']].head()

,content,cleaned_content_multi
0,Twice installing and uninstalling. Error all t...,twice installing and uninstalling error all th...
1,Very poor UI/UX. Took several times to scan ID...,very poor ui ux took several times scan id car...
2,i hope you're working on it cause this app is ...,i hope you working on cause this app is awful ...
3,I installed this apps because got informed tha...,i installed this apps because got informed tha...
4,"This is a very stupid app, when login with my ...",this is very stupid app when login with my pas...


In [17]:
filename = 'hasil/001_1_review_bank_digital_googleplaystore_cleaned_text_preprocessing.csv'
df.to_csv(filename, index=False)